In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os

In [2]:
prototxtPath= os.sep.join([r'./deploy.prototxt'])
weightsPath=os.sep.join([r'./res10_300x300_ssd_iter_140000.caffemodel'])

In [3]:
prototxtPath

'./deploy.prototxt'

In [4]:
net=cv2.dnn.readNet(prototxtPath, weightsPath)

In [5]:
model=load_model('mobilenet_v2.h5')

In [6]:
image=cv2.imread(r'./sample/istockphoto-1272058180-170667a.jpg')

In [7]:
image

array([[[215, 215, 215],
        [215, 215, 215],
        [215, 215, 215],
        ...,
        [215, 215, 215],
        [215, 215, 215],
        [215, 215, 215]],

       [[215, 215, 215],
        [215, 215, 215],
        [215, 215, 215],
        ...,
        [215, 215, 215],
        [215, 215, 215],
        [215, 215, 215]],

       [[215, 215, 215],
        [215, 215, 215],
        [215, 215, 215],
        ...,
        [215, 215, 215],
        [215, 215, 215],
        [215, 215, 215]],

       ...,

       [[215, 215, 215],
        [215, 215, 215],
        [215, 215, 215],
        ...,
        [215, 215, 215],
        [215, 215, 215],
        [215, 215, 215]],

       [[215, 215, 215],
        [215, 215, 215],
        [215, 215, 215],
        ...,
        [215, 215, 215],
        [215, 215, 215],
        [215, 215, 215]],

       [[215, 215, 215],
        [215, 215, 215],
        [215, 215, 215],
        ...,
        [215, 215, 215],
        [215, 215, 215],
        [215, 215, 215]]

In [8]:
(h,w)=image.shape[:2]
(h,w)

(339, 508)

In [9]:
blob=cv2.dnn.blobFromImage(image,1.0,(300,300),(104.0,177.0,123.0))

In [10]:
blob.shape

(1, 3, 300, 300)

In [11]:
net.setInput(blob)
detection=net.forward()

In [12]:
detection

array([[[[0.        , 1.        , 0.9998481 , ..., 0.20071928,
          0.6027807 , 0.6900956 ],
         [0.        , 1.        , 0.1243332 , ..., 4.0075474 ,
          4.840796  , 4.9883432 ],
         [0.        , 1.        , 0.11949161, ..., 4.0088034 ,
          0.8281528 , 4.9902306 ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]]]], dtype=float32)

In [13]:
for i in range(0,detection.shape[2]):
    confidence=detection[0,0,i,2]
    if confidence>0.5:
        box=detection[0,0,i,3:7]*np.array([w,h,w,h])
        (startX, startY, endX, endY)=box.astype('int')
        (startX,startY)=(max(0,startX),max(0,startY))
        (endX,endY)=(min(w-1,endX), min(h-1,endY))
        face=image[startY:endY, startX:endX]
        face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        face=cv2.resize(face,(224,224))
        face=img_to_array(face)
        face=preprocess_input(face)
        face=np.expand_dims(face, axis=0)
        (mask,withoutMask)=model.predict(face)[0]
        label='Mask' if mask>withoutMask else 'No Mask'
        color=(0,255,0) if label=='Mask' else (0,0,255)
        label="{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
        cv2.putText(image, label,(startX,startY-10),cv2.FONT_HERSHEY_DUPLEX,0.45,color,2)
        cv2.rectangle(image,(startX,startY),(endX,endY),color,2)

cv2.imshow('Output',image)
cv2.waitKey(0)
cv2.destroyAllWindows()

1/1 [==============================] - 1s 687ms/step
